# Basic CNN

In [3]:
import tensorflow.keras as keras
import pandas as pd

In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Dense,
    Conv2D,
    MaxPool2D,
    Flatten,
    Dropout,
    BatchNormalization,
)

model = Sequential()
model.add(Conv2D(75, (3, 3), strides=1, padding="same", activation="relu", 
                 input_shape=(28, 28, 1)))
model.add(BatchNormalization())
model.add(MaxPool2D((2, 2), strides=2, padding="same"))
model.add(Conv2D(50, (3, 3), strides=1, padding="same", activation="relu"))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(MaxPool2D((2, 2), strides=2, padding="same"))
model.add(Conv2D(25, (3, 3), strides=1, padding="same", activation="relu"))
model.add(BatchNormalization())
model.add(MaxPool2D((2, 2), strides=2, padding="same"))
model.add(Flatten())
model.add(Dense(units=512, activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(units=38, activation="softmax"))

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 28, 28, 75)        750       
_________________________________________________________________
batch_normalization_6 (Batch (None, 28, 28, 75)        300       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 14, 14, 75)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 14, 14, 50)        33800     
_________________________________________________________________
dropout_4 (Dropout)          (None, 14, 14, 50)        0         
_________________________________________________________________
batch_normalization_7 (Batch (None, 14, 14, 50)        200       
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 7, 7, 50)         

## Data Augmentation

In [64]:
# original:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
    zoom_range=0.1,  # Randomly zoom image
    width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=True,  # randomly flip images horizontally
    vertical_flip=False, # Don't randomly flip images vertically
)  

In [31]:
# new:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
                    rescale=1./255,
                    featurewise_center=False,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=False,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    zca_whitening=False,  # apply ZCA whitening
    rotation_range=45,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=True,  # randomly flip images
    vertical_flip=False
                    )

In [ ]:

BATCH_SIZE = 64
EPOCHS = 5

TRAIN_DIR = '/kaggle/input/new-plant-diseases-dataset/train'
TEST_DIR = '/kaggle/input/new-plant-diseases-dataset/test'

train_generator = ImageDataGenerator().flow_from_directory(TRAIN_DIR, batch_size=BATCH_SIZE, target_size = (28, 28))
test_generator = ImageDataGenerator().flow_from_directory(TEST_DIR, batch_size=BATCH_SIZE, target_size = (28, 28))

import tensorflow as tf

with tf.device("/device:GPU:0"):
    model.fit(
            train_generator,
            steps_per_epoch=10901//BATCH_SIZE,
            epochs=EPOCHS,
            validation_data=test_generator,
            validation_steps=2698//BATCH_SIZE
            )

# Separate out our target values
y_train = train_df['label']
y_valid = valid_df['label']
del train_df['label']
del valid_df['label']

# Separate our our image vectors
x_train = train_df.values
x_valid = valid_df.values

# Turn our scalar targets into binary categories
num_classes = 24
y_train = keras.utils.to_categorical(y_train, num_classes)
y_valid = keras.utils.to_categorical(y_valid, num_classes)

# Normalize our image data
x_train = x_train / 255
x_valid = x_valid / 255

# Reshape the image data for the convolutional network
x_train = x_train.reshape(-1,28,28,1)
x_valid = x_valid.reshape(-1,28,28,1)

In [66]:
datagen.fit(x_train)

In [67]:
model.compile(loss='categorical_crossentropy', metrics=['accuracy'])

In [2]:
model.fit(img_iter,
          epochs=10,
          steps_per_epoch=len(x_train)/batch_size, # Run same number of steps we would if we were not using a generator.
          validation_data=(x_valid, y_valid))


Epoch 1/10
857/857 [==============================] - 11s 12ms/step - loss: 0.7633 - accuracy: 0.5911 - val_loss: 0.7521 - val_accuracy: 0.5897
Epoch 2/10
857/857 [==============================] - 10s 12ms/step - loss: 0.7785 - accuracy: 0.6122 - val_loss: 0.7323 - val_accuracy: 0.5422
Epoch 3/10
857/857 [==============================] - 10s 12ms/step - loss: 0.7794 - accuracy: 0.6154 - val_loss: 0.6044 - val_accuracy: 0.6551
Epoch 4/10
857/857 [==============================] - 10s 12ms/step - loss: 0.7087 - accuracy: 0.7013 - val_loss: 0.5521 - val_accuracy: 0.7001
Epoch 5/10
857/857 [==============================] - 10s 12ms/step - loss: 0.6479 - accuracy: 0.7771 - val_loss: 0.5828 - val_accuracy: 0.7323
Epoch 6/10
857/857 [==============================] - 11s 12ms/step - loss: 0.6535 - accuracy: 0.7896 - val_loss: 0.6077 - val_accuracy: 0.7199
Epoch 7/10
857/857 [==============================] - 10s 11ms/step - loss: 0.5644 - accuracy: 0.8023 - val_loss: 0.5233 - val_accuracy